In [ ]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np

from scipy.fft import fft, fftfreq

import networkx as nx
import cv2

from main import CircleCalibWindow, EllipseCalibWindow, resize, showImg

from skimage import (
    color, feature, filters, measure, morphology, segmentation, util
)

# Process Image

In [ ]:
img = cv2.imread("slime_mould/HELP1_10_w_v/271023_1053/KCL_20231027_110612.jpg")

## Crop and Perspective Transform

In [ ]:
calibWin = EllipseCalibWindow("Ellipse Calibration", resize(im))
calibWin.displayWindow()

In [ ]:
centerX, centerY, radius = CircleCalibWindow.getCircle()

## Threshold

In [ ]:
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
h,s,v = cv2.split(hsv)
showImg(resize(h)[0])

In [ ]:
showImg(resize(img)[0])

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
iterations = 2
clahe = cv2.createCLAHE(clipLimit=1.2, tileGridSize=(12,12))
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
tophat1 = gray.copy()
for i in range(iterations):
    cl1 = clahe.apply(tophat1)
    tophat1 = cv2.morphologyEx(cl1, cv2.MORPH_TOPHAT, cl1)

In [ ]:
thresholds = filters.threshold_multiotsu(s, classes=3)
regions = np.digitize(s, bins=thresholds)

fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
ax[0].imshow(s)
ax[0].set_title('Original')
ax[0].axis('off')
ax[1].imshow(regions)
ax[1].set_title('Multi-Otsu thresholding')
ax[1].axis('off')
plt.show()


In [ ]:
showImg(tophat1)

# Network

In [ ]:
G = nx.Graph([(0, 1), (1, 2), (5, 6), (3, 4)])
nx.node_connected_component(G, 0)

In [ ]:
import networkx as nx
import numpy as np

def skeleton_image_to_graph(skeIm, connectivity=2):
    assert(len(skeIm.shape) == 2)
    skeImPos = np.stack(np.where(skeIm))
    skeImPosIm = np.zeros_like(skeIm, dtype=np.int)
    skeImPosIm[skeImPos[0], skeImPos[1]] = np.arange(0, skeImPos.shape[1])
    g = nx.Graph()
    if connectivity == 1:
        neigh = np.array([[0, 1], [0, -1], [1, 0], [-1, 0]])
    elif connectivity == 2:
        neigh = np.array([[0, 1], [0, -1], [1, 0], [-1, 0], [1, 1], [1, -1], [-1, 1], [-1, -1]])
    else:
        raise ValueError(f'unsupported connectivity {connectivity}')
    for idx in range(skeImPos[0].shape[0]):
        for neighIdx in range(neigh.shape[0]):
            curNeighPos = skeImPos[:, idx] + neigh[neighIdx]
            if np.any(curNeighPos<0) or np.any(curNeighPos>=skeIm.shape):
                continue
            if skeIm[curNeighPos[0], curNeighPos[1]] > 0:
                g.add_edge(skeImPosIm[skeImPos[0, idx], skeImPos[1, idx]], skeImPosIm[curNeighPos[0], curNeighPos[1]], weight=np.linalg.norm(neigh[neighIdx]))
    g.graph['physicalPos'] = skeImPos.T
    return g


# Animations

## FFT

In [ ]:
def plotFFT(mode):
    amplitude = 0.2
    radius = 1
    N = 10000

    theta = np.linspace(0, 2*np.pi, N)
    r = radius + amplitude*np.sin(mode*theta)
    plt.plot(theta, r)
    plt.show()

    x = r*np.cos(theta)
    y = r*np.sin(theta)
    plt.plot(x, y)
    plt.gca().set_aspect('equal')
    plt.show()

    nRepeats = 20
    thetaRepeat = np.linspace(0, nRepeats*2*np.pi, N*nRepeats)
    rRepeat = np.tile(r, nRepeats)
    # plt.plot(thetaRepeat, rRepeat)
    # plt.show()

    yf = fft(rRepeat)
    xf = fftfreq(N*nRepeats, np.diff(thetaRepeat)[0])[:N*nRepeats//2]
    plt.plot(2*np.pi*xf[1:N*nRepeats//2], 2.0/N*nRepeats * np.abs(yf[1:N*nRepeats//2]))
    plt.xlim(2, 6)
    plt.xticks([2, 3, 4, 5, 6])
    plt.show()

In [ ]:
plotFFT(3)

In [ ]:
plotFFT(4)

In [ ]:
amplitude = 0.2
radius = 1
mode = 4
N = 10000

theta = np.linspace(0, 2*np.pi, N)
r = radius + amplitude*np.sin(mode*theta)
plt.plot(theta, r)
plt.show()

x = r*np.cos(theta)
y = r*np.sin(theta)
plt.plot(x, y)
plt.gca().set_aspect('equal')
plt.show()

nRepeats = 20
thetaRepeat = np.linspace(0, nRepeats*2*np.pi, N*nRepeats)
rRepeat = np.tile(r, nRepeats)

plt.plot(thetaRepeat, rRepeat)
plt.show()

yf = fft(rRepeat)
xf = fftfreq(N*nRepeats, np.diff(thetaRepeat)[0])[:N*nRepeats//2]
plt.plot(2*np.pi*xf[1:N*nRepeats//2], 2.0/N*nRepeats * np.abs(yf[1:N*nRepeats//2]))
plt.xlim(0, 10)